In [1]:
import pandas as pd

In [2]:
def preprocess_df(location):
    df = pd.read_parquet(location)
    # df=df.head(50000)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration=df.duration.apply(lambda tr: tr.total_seconds()/60)
    # df=df.drop(df[df.duration < 0].index)
    # df=df[(df.duration >= 1) & (df.duration <= 60)]
    df.PULocationID = df.PULocationID.astype(str)
    df.DOLocationID = df.DOLocationID.astype(str)
    df['travel_map'] = df.PULocationID + "_" + df.DOLocationID

    return df


In [3]:
df_jan=preprocess_df("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
df_feb=preprocess_df("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

In [ ]:
len(df_jan.columns[:-2])

19

: 

In [ ]:
df_jan['duration']